# Quantum portfolio optimization (DWave)

D-Wave's approach ti quantum computing so far is based on quantum annealers, so they perfectly fit into what we have been doing until now. 

Let us choose a set of assets to evaluate.

In [3]:
from quanvia.finance import utils

# Get first 25 elements of USDT list
num_assets = 25
asset_list = utils.get_binance_assets()
asset_list = [x for x in asset_list if x.endswith("USDT")]
asset_list = asset_list[:num_assets]

# Get the data
df = utils.get_binance_data(asset_list)
mu, sigma = utils.get_exp_cov(df)
mu_vals = list(mu.values())
num_assets = len(mu) # Effective assets

# Budget
budget = 2

In [1]:
from pyqubo import Array, Placeholder, Constraint
import neal

# A simulated sampler to be used locally
sampler = neal.SimulatedAnnealingSampler()

In [5]:
x = Array.create('x', shape=num_assets, vartype='BINARY')

# Profit generated by each asset individually
H_linear_profit = 0.0
for i in range(num_assets):
    H_linear_profit += Constraint(
        mu_vals[i] * x[i], label='profit({})'.format(i)
    )

# Risk obtained from the covariance matrix
H_quadratic_profit = 0.0
for i in range(num_assets):
    for j in range(i + 1, num_assets):
        H_quadratic_profit += Constraint(
            sigma[i][j] * x[i] * x[j], label='risk({}, {})'.format(i, j)
        )

# Constraint.
H_linear_weight = 0.0
for i in range(num_assets):
    H_linear_weight += Constraint(x[i], label='slot({})'.format(i))

# Build model.
alpha = Placeholder('alpha')
H = - (H_linear_profit + H_quadratic_profit) + alpha * (budget - H_linear_weight)**2
model = H.compile()

# Set the Lagrange multiplier to restring the budget
feed_dict = {'alpha': 300.0}

# Transform to QUBO.
qubo, offset = model.to_qubo(feed_dict=feed_dict)

# Run.
num_reads = 1000
response = sampler.sample_qubo(qubo, num_reads=num_reads)
response = response.aggregate()

Selected vehicles: [2, 5]


In [9]:
# Selected assets.
best_sample = response.first
solution = [
    int(item[2:-1])  # As each variable has the shape 'x[i]', we will just keep 'i'.
    for item, item_selected in best_sample.sample.items()
    if item_selected
]
print('Selected assets: {}'.format(solution))

# Expected profits.
expected_profit = 0.0
for i in solution:
    expected_profit += mu_vals[i]
print('Expected profits: {:,}'.format(float(expected_profit)))

Selected assets: [2, 5]
Expected profits: 0.015824602713604422


In [10]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import FixedEmbeddingComposite
import minorminer
import dimod

# Instanciate Sampler
dwave_sampler = DWaveSampler()

# Construct a problem
bqm = dimod.BinaryQuadraticModel(qubo, dimod.BINARY)

# Get the edge list
target_edgelist = dwave_sampler.edgelist
# And source edge list on the BQM quadratic model
source_edgelist = list(bqm.quadratic)
# Find the embeding
embedding = minorminer.find_embedding(source_edgelist, target_edgelist)
sampler = FixedEmbeddingComposite(dwave_sampler, embedding)

In [11]:
# Sample
response = sampler.sample_qubo(qubo, num_reads=num_reads)
response = response.aggregate()

# Selected assets.
best_sample = response.first
solution = [
    int(item[2:-1])  # As each variable has the shape 'x[i]', we will just keep 'i'.
    for item, item_selected in best_sample.sample.items()
    if item_selected
]
print('Selected assets: {}'.format(solution))

# Expected profits.
expected_profit = 0.0
for i in solution:
    expected_profit += mu_vals[i]
print('Expected profits: {:,}'.format(float(expected_profit)))

Selected assets: [2, 5]
Expected profits: 0.015824602713604422


![dwave](../assets/dwave.PNG)

# Additional references

* https://www.dwavesys.com/media/5qahck2o/multiverse_case_study_v7-1.pdf
* https://arxiv.org/pdf/2012.01121.pdf